In [5]:
import numpy as np
import plotly.express as px
import pandas as pd
from instance_mongodb import instance_mongodb_sei

In [37]:
db = instance_mongodb_sei(project="mlts")
collection = db.rudorff_lilienfeld_calculation

cursor = collection.find(
    {"task_label": "frequency calculation"},
    {"calcs_reversed.frequencies": 1, "tags.reaction_name": 1, "tags.rxn_number": 1},
)

df = pd.DataFrame(columns=["reaction_name", "rxn_number",  "num_imag_frequencies"])

for doc in cursor:
    frequencies = doc["calcs_reversed"][0]["frequencies"]
    frequencies = np.array(frequencies)
    num_imag_frequencies = np.sum(frequencies < 0)
    df = pd.concat(
        [
            df,
            pd.DataFrame(
                {
                    "reaction_name": doc["tags"]["reaction_name"],
                    "rxn_number": doc["tags"]["rxn_number"],
                    "num_imag_frequencies": num_imag_frequencies,
                },
                index=[0],
            ),
        ]
    )

df = df.reset_index(drop=True)

# Make a histogram of the number of imaginary frequencies for the 
# sn2 and e2 reactions on the same plot.
fig = px.histogram(
    df,
    x="num_imag_frequencies",
    color="reaction_name",
    barmode="overlay",
    # histnorm="probability density",
    nbins=10,
    opacity=0.75,
    marginal="box",
    title="Number of imaginary frequencies for SN2 and E2 reactions",
    labels={"num_imag_frequencies": "Number of imaginary frequencies"},
)
fig.show()
